# Vehicle Route Optimization

This solution uses the openrouteservice library

In [11]:
import os
from dotenv import load_dotenv

load_dotenv(".env")

api_key = os.getenv('ORS_KEY')

In [12]:
import folium
import openrouteservice as ors
import math


In [17]:
coords = [[-89.50931047510767, 43.09556864914019], # depot
          [-89.50747701928485, 43.10058814207571], # school
          [-89.53681317510716, 43.10481616845111], # rink
          [-89.44951351743772, 43.0766193182772], # store
          [-89.40896345976672, 43.07628461311108] # observatory             
          ]

m = folium.Map(location=list([43.09557209600966, -89.50436892925059]), tiles="cartodbpositron", zoom_start=13)
for coord in coords:
    folium.Marker(location=list(reversed(coord))).add_to(m)
m


In [38]:
client = ors.Client(key=api_key)

m = folium.Map(location=list([43.09557209600966, -89.50436892925059]), tiles="cartodbpositron", zoom_start=13)
for coord in coords:
    folium.Marker(location=list(reversed(coord))).add_to(m)

vehicle_start = [-89.50931047510767, 43.09556864914019]
folium.Marker(location=list(reversed(vehicle_start)), icon=folium.Icon(color="red")).add_to(m)


vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[3]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[3])
]
jobs = [ors.optimization.Job(id=index, location=coords, amount=[1]) for index, coords in enumerate(coords)]
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)
line_colors = ['green', 'orange', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(m)

m